In [1]:
import pandas as pd 
import numpy as np
import requests
import json
from pprint import pprint
from sqlalchemy import create_engine
import psycopg2

In [2]:
#Read in CSV file
top_100 = "Resources/top_100_all_time.csv"
top_100_df = pd.read_csv(top_100)
top_100_df.head()

,Index,Position,ISBN,Title,Author,Imprint,Publisher Group,Volume,Value,RRP,ASP,Binding,Publ Date,Product Class
0,1,1,9780552149518,"Da Vinci Code,The","Brown, Dan",Corgi Books,Transworld Grp,"4,522,025","£22,857,837.53",£7.99,£5.05,Paperback,1 Mar 2004,"F2.1 Crime, Thriller & Adventure"
1,3,2,9780747532743,Harry Potter and the Philosopher's Stone#,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,844,316","£19,853,187.43",£6.99,£5.22,Paperback,26 Jun 1997,Y2.1 Children's Fiction
2,7,3,9780747538486,Harry Potter and the Chamber of Secrets#,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,184,492","£16,224,021.98",£6.99,£5.07,Paperback,1 Apr 1999,Y2.1 Children's Fiction
3,6,4,9780552150736,Angels and Demons#,"Brown, Dan",Corgi Books,Transworld Grp,"3,096,850","£15,537,324.84",£7.99,£5.05,Paperback,1 Jul 2003,"F2.1 Crime, Thriller & Adventure"
4,2,5,9780747551003,Harry Potter and the Order of the Phoenix,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,043,226","£33,925,431.19",£16.99,£11.15,Hardback,21 Jun 2003,Y2.1 Children's Fiction


In [3]:
# Pull certain columns
top_100_cols = ["Position","ISBN", "Title", "Author", "Volume", "Publ Date"]
top_100_transformed= top_100_df[top_100_cols].copy()

In [4]:
# Rename columns
top_100_transformed = top_100_transformed.rename(columns={"Position": "position",
                                                          "ISBN": "isbn",
                                                          "Title": "title",
                                                          "Author": "author",
                                                          "Volume": "volume",
                                                          "Publ Date": "publish_date"})

top_100_transformed

,position,isbn,title,author,volume,publish_date
0,1,9780552149518,"Da Vinci Code,The","Brown, Dan","4,522,025",1 Mar 2004
1,2,9780747532743,Harry Potter and the Philosopher's Stone#,"Rowling, J. K.","3,844,316",26 Jun 1997
2,3,9780747538486,Harry Potter and the Chamber of Secrets#,"Rowling, J. K.","3,184,492",1 Apr 1999
3,4,9780552150736,Angels and Demons#,"Brown, Dan","3,096,850",1 Jul 2003
4,5,9780747551003,Harry Potter and the Order of the Phoenix,"Rowling, J. K.","3,043,226",21 Jun 2003
...,...,...,...,...,...,...
95,96,9780439993586,"Amber Spyglass,The:His Dark Materials S.","Pullman, Philip","653,615",14 Sep 2001
96,97,9780552771108,Can You Keep a Secret?,"Kinsella, Sophie","652,788",17 Mar 2003
97,98,9780552997034,Down Under,"Bryson, Bill","646,515",6 Aug 2001
98,99,9780099506928,"Spot of Bother,A","Haddon, Mark","645,453",7 Jun 2007


In [5]:
# Drop duplicates 
top_100_transformed.drop_duplicates("isbn", inplace=True)

#Set index
top_100_transformed.set_index("position", inplace=True)

In [6]:
top_100_transformed

,isbn,title,author,volume,publish_date
position,,,,,
1,9780552149518,"Da Vinci Code,The","Brown, Dan","4,522,025",1 Mar 2004
2,9780747532743,Harry Potter and the Philosopher's Stone#,"Rowling, J. K.","3,844,316",26 Jun 1997
3,9780747538486,Harry Potter and the Chamber of Secrets#,"Rowling, J. K.","3,184,492",1 Apr 1999
4,9780552150736,Angels and Demons#,"Brown, Dan","3,096,850",1 Jul 2003
5,9780747551003,Harry Potter and the Order of the Phoenix,"Rowling, J. K.","3,043,226",21 Jun 2003
...,...,...,...,...,...
96,9780439993586,"Amber Spyglass,The:His Dark Materials S.","Pullman, Philip","653,615",14 Sep 2001
97,9780552771108,Can You Keep a Secret?,"Kinsella, Sophie","652,788",17 Mar 2003
98,9780552997034,Down Under,"Bryson, Bill","646,515",6 Aug 2001


In [7]:
# Create engine
connection_string = "{pg_user}:{pg_password}@localhost:5432/top_100_db"
engine = create_engine(f'postgresql://{connection_string}')

In [8]:
# Table name list
engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "{pg_user}"

(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [ ]:
# Dataframe to SQL
top_100_transformed.to_sql(name='top_books', con=engine, if_exists='append', index=False)